In [1]:
import pandas as pd
import numpy as np
from utils.load import load_data_methodology_random_cutting
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from utils.metrics import f1
from utils.tuner import CVTuner
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import joblib
import keras_tuner
import tensorflow as tf

In [2]:
df = pd.read_csv("../Metadata/data_curated.csv")
df.set_index("File", inplace=True)
df.head()

,Class
File,
S1V3C1M3A Clip166.npy,1
S1V1C1M1A Clip18.npy,1
S1V4C1M4A Clip241.npy,1
S1V4C1M4A Clip239.npy,1
S1V4C1M4A Clip245.npy,1


In [3]:
X_train, y_train, X_val, y_val = load_data_methodology_random_cutting(df,["M1"],30)

In [4]:
X_train.shape

(2250, 30, 1662)

In [5]:
y_train.shape

(2250, 6)

In [6]:
X_val.shape

(750, 30, 1662)

In [7]:
y_val.shape

(750, 6)

# Cross Validation

In [8]:
cv = MultilabelStratifiedKFold(n_splits=5)
joblib.dump(cv, './cv/cv.joblib')

['./cv/cv.joblib']

# Hyper Parameter Tunning Recurrent Neural Networks

In [9]:
def build_model_GRU(hp):
    
    model = Sequential()


    model.add(GRU(
        hp.Int('input_units',min_value=10,max_value=70,step=10),
        return_sequences=True,
        activation='relu', 
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal(),
        recurrent_dropout=0.1,
        input_shape=(30,1662)))
    
    model.add(GRU(
        hp.Int('GRU_2',min_value=10,max_value=70,step=10), 
        recurrent_dropout=0.1,
        return_sequences=False,
        activation='relu',
        kernel_initializer=tf.keras.initializers.GlorotNormal(),
        recurrent_initializer=tf.keras.initializers.Orthogonal()
        ))
    

    model.add(Dense(
        hp.Int(f"Dense_units_1",min_value=10,max_value=70,step=10),
        activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
        ))
    
    model.add(Dense(6, activation='softmax'))

    opt = tf.keras.optimizers.Nadam(clipnorm = 1.0)
    model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=[f1])

    return model

In [10]:
tuner_GRU = CVTuner(
                    data_cv= cv,
                    goal = 'f1',
                    hypermodel=build_model_GRU,
                    oracle=keras_tuner.oracles.BayesianOptimizationOracle(
                    objective=keras_tuner.Objective('f1',direction = "max"),
                        max_trials=5                  
                    ),
                    directory='./experiments/',
                    project_name = 'GRU',
                    overwrite=True
                    )

In [11]:
tuner_GRU.search(X_train, y_train, 64, epochs=50)

Trial 5 Complete [00h 09m 48s]
f1: 0.8766876459121704

Best f1 So Far: 0.9364292025566101
Total elapsed time: 01h 17m 37s


In [15]:
df_GRU = pd.DataFrame(tuner_GRU.trial_scores)

In [16]:
df_GRU

,trial_id,hyperparameters,f1,f1_std,precision,precision_std,recall,recall_std
0,0,"{'input_units': 70, 'GRU_2': 20, 'Dense_units_...",0.904232,0.000000,0.906250,0.000000,0.902222,0.000000
1,0,"{'input_units': 70, 'GRU_2': 20, 'Dense_units_...",0.956522,0.026145,0.932991,0.026741,0.927778,0.025556
2,0,"{'input_units': 70, 'GRU_2': 20, 'Dense_units_...",0.935123,0.021464,0.935808,0.022194,0.928148,0.020873
3,0,"{'input_units': 70, 'GRU_2': 20, 'Dense_units_...",0.951002,0.020336,0.940137,0.020632,0.933333,0.020184
4,0,"{'input_units': 70, 'GRU_2': 20, 'Dense_units_...",0.935268,0.018198,0.940002,0.018456,0.932889,0.018075
5,1,"{'input_units': 60, 'GRU_2': 10, 'Dense_units_...",0.928251,0.000000,0.936652,0.000000,0.920000,0.000000
6,1,"{'input_units': 60, 'GRU_2': 10, 'Dense_units_...",0.943946,0.007848,0.944570,0.007919,0.927778,0.007778
7,1,"{'input_units': 60, 'GRU_2': 10, 'Dense_units_...",0.926009,0.007980,0.941176,0.008052,0.924444,0.007909
8,1,"{'input_units': 60, 'GRU_2': 10, 'Dense_units_...",0.852643,0.035363,0.921714,0.034424,0.903889,0.036256
9,1,"{'input_units': 60, 'GRU_2': 10, 'Dense_units_...",0.948775,0.034763,0.927550,0.032928,0.912444,0.036666


In [17]:
df_GRU.to_csv("./experiments/GRU.csv")

In [18]:
df_GRU.to_excel("./experiments/GRU.xlsx")

In [ ]:
#disp = confusion_matrix_plot(y_val, y_pred)
#disp.plot()